# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

ETL pipeline - 
Extract, 
Transform (python to make the data what we want), 
Load (use sql so we have the data for us to access and view)

business id foreign key for business??

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


In [49]:
DB_NAME = 'yelp_practice'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [36]:
# import sqlite3
import pandas as pd

In [37]:
# conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
# cur = conn.cursor()

In [50]:
cursor.execute("""SELECT * FROM businesses WHERE avg_rating = 4""")
df = pd.DataFrame(cursor.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in cursor.description]
df.head()

,restaurant_id,name,avg_rating,price
0,-0YTqofxupHzmd9SbKhjXw,Los Verdes,4,$
1,-PQ_yAucEcu3xQxsKzJWtw,5 Napkin Burger,4,$$
2,-V0rOWj65nEwvC3uf4wWtw,AwesomeSauce,4,$$
3,0QXk9toOB4m3c2jfCX-a7g,Pelicana Chicken,4,$$$
4,1ExwosnCMtb7D3SF5ovo6w,The Pomeroy,4,$$


In [ ]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(1999, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [1]:
## Connect to DB server on AWS
!pip install mysql-connector

In [8]:
import mysql.connector
import config

In [10]:
config.password

'nma18173'

In [6]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

cursor = cnx.cursor() #creating workspace

The connection has been established, and cursor(workspace) has been made!

In [13]:
## Create new DB 

import mysql.connector
from mysql.connector import errorcode

In [14]:
db_name = 'yelp_practice'

def create_database(cursor, database):  #your cursor  and  name of your database
    #connect to database instance
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [17]:
#Creating the database now
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Create tables now

In [19]:
DB_NAME = 'yelp_practice'
TABLES = {}

In [20]:
# Create a table for the Businesses

TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    "  restaurant_id varchar(24) NOT NULL,"                                                  
    "  name varchar(24),"
    "  avg_rating int(2),"
    "  price varchar(3),"
    "  PRIMARY KEY (restaurant_id)"
    ") ENGINE=InnoDB")


In [21]:
# Create a table for the reviews

TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  review_id varchar(33) NOT NULL,"
    "  customer_id varchar(33) NOT NULL,"
    "  review_rating int(11),"
    "  review_date date,"
    "  restaurant_id varchar(33) NOT NULL,"
    "  PRIMARY KEY (restaurant_id,review_id), KEY restaurant_id (restaurant_id),"
    "  CONSTRAINT reviews_ibfk_1 FOREIGN KEY (restaurant_id) "
    "     REFERENCES businesses (restaurant_id) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [121]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

Creating the tables now.

In [26]:


for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()



Creating table businesses: OK
Creating table reviews: OK


## Part 2: Create ETL pipeline for the business data from the API

In [7]:
import time
import requests
import json

In [55]:
#State universal variables
url1 = 'https://api.yelp.com/v3/businesses/search'
# url2 = 
api_key = 'WMgB0oJS5hb-hZqNk4SxrM5YOv-WVzabBsyO1InWv-5_esbUAR2pKavVcXBRWiLYcLmrVukepWZGwyhZUP2VvSMhNpMuUonlpLAiUJlqlHFW3nCgP0LzzzC_3Rk6XXYx'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
client_id = 'ZHIPHezpOPB8hoFciU-7rQ'
term = 'burger'
location = 'Astoria NY'
url_params_bus = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
response = requests.get(url1, headers=headers, params=url_params_bus)


In [51]:
# write a function to make a call to the API
#BUSINESS YELP CALL

def business_yelp_call(url_params, api_key):
    response = requests.get(url1, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [126]:
#REVIEW YELP CALL

def review_yelp_call(url_params, api_key, business_id):
    url2 = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    response = requests.get(url2, headers=headers, params=url_params)
    
    data = response.json()['reviews']
    return data

business_id = '-0YTqofxupHzmd9SbKhjXw'
review_yelp_call(url_params_bus, api_key, business_id)[0]['user']['id']


'K54V9WW1Udbd3kJ6dWq5Sg'

In [127]:
# business_yelp_call(url_params_bus, api_key)[0]

In [78]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

def all_results_businesses(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 200:
        url_params['offset'] = cur
        results.extend(business_yelp_call(url_params_bus, api_key)) 
        #parse data and send to db instead of results list?
        add_businesses(business_yelp_call(url_params_bus,api_key))
        time.sleep(1) #Wait a second
        cur += 50
    return len(results), results



In [125]:
# all_results_businesses(url_params_bus, api_key)

In [118]:
#QUERY BUSINESS AND REVIEW DATA

def add_businesses(data):
    for business in data:
        if business['price']:
            cursor.execute('''
            INSERT IGNORE INTO businesses (restaurant_id, name, avg_rating, price)
               VALUES( %s, %s, %s, %s)''', (business['id'], business['name'], business['rating'], business['price']))
        else:
            cursor.execute('''
            INSERT IGNORE INTO businesses (restaurant_id, name, rating, price)
               VALUES( %s, %s, %s, %s)''', (business['id'], business['name'], business['rating'], None))
        
        business_id = business['id']
        add_reviews(review_yelp_call(url_params_bus, api_key, business_id))
        cnx.commit()
        #yelp_call_reviews(business['id'])

In [70]:
# Write a function to take your parsed data and insert it into the DB

# def add_businesses(data):
#     for business in data:
#         c.execute('''INSERT IGNORE INTO businesses
#                     (restaurant_id, name, avg_rating, price)
#                     VALUES( %s, %s, %s, %s)''', 
#                   (business['id'], business['name'], business['rating'], business['price']))
    
#     cnx.commit()





In [123]:
#QUERY YELP REVIEWS TO DATA BASE

def add_reviews(data):
    for review in data:
        cursor.execute('''INSERT IGNORE INTO reviews
                    (review_id, customer_id, review_rating, review_date, restaurant_id)
                    VALUES( %s, %s, %s, %s, %s)''', 
                  (review['id'], review['user']['id'], review['rating'], review['time_created'], business_id))
    
    cnx.commit()
    


In [129]:
cursor.close()
cnx.close()

## Part 3: Create ETL pipeline for the review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that take a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key)) 
        #parse data and send to db instead of results list?
        time.sleep(1) #Wait a second
        cur += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```